In [ ]:
# Merge features from various datasets, including creating new ones.
# This file currently includes merging population and population density,
# income, 

In [1]:
# Merge yelp business dataset with population & population density from simple maps dataset

import pandas as pd

yelpBus_df = pd.read_json('/Users/james/TIM147/yelp_dataset/yelp_academic_dataset_business.json',lines=True) # Import yelp business data
simple_df = pd.read_csv('/Users/james/Tim147/simplemaps_uszips_basicv1/uszips.csv',) # Import simple maps data
income_df = pd.read_csv('/Users/james/Tim147/income_dataset/kaggle_income.csv', encoding='iso-8859-1 ',)

In [3]:
yelpBus_df['postal_code'] = pd.to_numeric(yelpBus_df['postal_code'], errors='coerce') # Convert postal code from object to float64
yelpBus_df['postal_code'] = yelpBus_df['postal_code'].astype('Int64') # Convert postal code from float64 to Int64, with capital I so that it works with nan values

In [4]:
simple_df = simple_df[['zip','population','density']]
simple_df = simple_df.rename(columns = {'zip': 'postal_code', 'density': 'pop_density'})

In [5]:
merged_df = pd.merge(yelpBus_df, simple_df, on='postal_code', how='left') # Merge 

In [6]:
# Merge income on nearest latitude & longitude

import geopandas as gpd
from geopandas.tools import sjoin_nearest # Spatial join
from shapely.geometry import Point # 

income_df = income_df[['Lat','Lon','Median']]
income_df = income_df.rename(columns = {'Median': 'median_income'})

In [7]:
left_geo = [Point(xy) for xy in zip(merged_df['latitude'],merged_df['longitude'])]
right_geo = [Point(xy) for xy in zip(income_df['Lat'],income_df['Lon'])]

In [8]:
merged_gdf = gpd.GeoDataFrame(merged_df, geometry=left_geo)
income_gdf = gpd.GeoDataFrame(income_df, geometry=right_geo)

In [9]:
sjoined_gdf = sjoin_nearest(merged_gdf, income_gdf, how='left')

In [10]:
sjoined_df = sjoined_gdf.drop(['geometry','index_right','Lat','Lon'], axis=1)

In [ ]:
sjoined_df.to_csv('yelp_business_plus.csv')

In [14]:
sjoined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162988 entries, 0 to 150345
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   business_id    162988 non-null  object 
 1   name           162988 non-null  object 
 2   address        162988 non-null  object 
 3   city           162988 non-null  object 
 4   state          162988 non-null  object 
 5   postal_code    157354 non-null  Int64  
 6   latitude       162988 non-null  float64
 7   longitude      162988 non-null  float64
 8   stars          162988 non-null  float64
 9   review_count   162988 non-null  int64  
 10  is_open        162988 non-null  int64  
 11  attributes     148067 non-null  object 
 12  categories     162873 non-null  object 
 13  hours          137871 non-null  object 
 14  population     156234 non-null  float64
 15  pop_density    156234 non-null  float64
 16  median_income  162988 non-null  int64  
 17  locations      162988 non-null  in

In [ ]:
# Create the feature for size of chain.

In [17]:
# Regularize the strings of the business names
sjoined_df['name'] = sjoined_df['name'].str.lower() # Make all lowercase
sjoined_df['name'] = sjoined_df['name'].str.strip() # Strip leading and following spaces
sjoined_df['name'] = sjoined_df['name'].str.replace('[^a-zA-Z0-9]','', regex=True)

# Calculate the number of locations for each business and create a column for it.
sjoined_df['locations'] = sjoined_df.groupby('name')['name'].transform('count')

In [ ]:
sjoined_df.head(10)

In [ ]:
sjoined_df.tail(20)

In [ ]:
# Validated that the number of locations in the dataset is actually correct.
check_location_count = sjoined_df[sjoined_df['name'] == 'arbys']
check_location_count

In [ ]:
# The maximum number of locations in the dataset is 783.
check_max_locations = sjoined_df.sort_values(by='locations')
check_max_locations

In [44]:
bin_exploration = sjoined_df[sjoined_df['locations'] == 187]
bin_exploration

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,population,pop_density,median_income,locations
2254,sNDXKUS14_xXdD0fJdw23g,jimmyjohns,"6260 Intech Commons Rd, Ste C",Indianapolis,IN,46278,39.880535,-86.271365,3.5,14,1,"{'GoodForMeal': 'None', 'RestaurantsReservatio...","Restaurants, Fast Food, Delis, Sandwiches","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",8932.0,261.0,22270,187
5406,YwxnKaMrUVtBygE8kagsVw,jimmyjohns,2402 N Meridian St,Indianapolis,IN,46208,39.800540,-86.157093,2.5,21,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Delis, Fast Food, Restaurants, Sandwiches","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",21908.0,1226.5,13263,187
5469,at5w9OCM1ex1Y4mWCQB2Bw,jimmyjohns,3142 Nameoki Rd,Granite City,IL,62040,38.719600,-90.122339,2.5,9,1,"{'RestaurantsGoodForGroups': 'True', 'BikePark...","Food, Food Delivery Services, Fast Food, Delis...","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",40912.0,396.0,45101,187
5644,pXr5ndDooMsfIxuczExK6A,jimmyjohns,2424 James L Redman Pkwy,Plant City,FL,33566,27.988557,-82.121514,3.0,13,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","Delis, Restaurants, Sandwiches, Fast Food","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",22415.0,365.6,36287,187
6457,1wC41D8PLqOldFPUVAZ3tA,jimmyjohns,3822 Laclede Ave,St. Louis,MO,63108,38.635655,-90.240701,2.5,43,1,"{'RestaurantsGoodForGroups': 'True', 'NoiseLev...","Food, Delis, Food Delivery Services, Sandwiche...","{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...",20291.0,3494.0,10949,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146972,Dve6VC2YIrKdzRsMKU4flw,jimmyjohns,"365 W 116th St, Ste 100",Carmel,IN,46032,39.955554,-86.163174,3.5,19,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","Fast Food, Restaurants, Delis, Sandwiches","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",50309.0,766.3,300000,187
147178,Ywo3yKN1Qqt9QHGoDeQ0Cg,jimmyjohns,5013 Crossings Cir,Mount Juliet,TN,37122,36.166661,-86.514879,3.0,12,0,"{'HasTV': 'False', 'BusinessAcceptsCreditCards...","Restaurants, Food, Fast Food, Food Delivery Se...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",62696.0,216.9,300000,187
147916,f4ouae_IyCaqGo7zERBXqQ,jimmyjohns,"7614 Hwy 70 S, Ste 605",Nashville,TN,37221,36.079461,-86.947977,4.0,8,1,"{'Ambience': 'None', 'RestaurantsReservations'...","Restaurants, Sandwiches, Fast Food, Delis","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",42232.0,330.5,67403,187
149694,a69pvnfQKMrQ4WXIMdMLCg,jimmyjohns,"475 Sparks Blvd, Ste. 121",Sparks,NV,89434,39.535404,-119.716089,2.5,78,1,"{'BusinessAcceptsCreditCards': 'True', 'Alcoho...","Restaurants, Delis, Fast Food, Sandwiches","{'Monday': '10:30-20:0', 'Tuesday': '10:30-20:...",27478.0,84.6,69325,187


In [59]:
'''Create intervals of chain size. 1-3 locations is considered not a chain
because it requires very little infrastructure and can be run by one business
owner. 4-20 locations is considered a small chain, as ACA chain restaurant regulations
begin at 20 locations. 21-50 is a medium chain, 51-100 a large, and 101+ a mega chain.
This may be biased by the fact that not all locations are included in the dataset,
especially for large and mega chains.'''

bins = [0, 3, 20, 50, 100, float('inf')]
labels = ['not', 'small', 'medium', 'large', 'mega']
sjoined_df['chain'] = pd.cut(sjoined_df['locations'], bins=bins, labels=labels, right=False)
sjoined_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,population,pop_density,median_income,locations,chain
0,Pns2l4eNsfO8kk83dixA6A,abbyrappoportlaccmq,"1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,31439.0,3790.5,57604,1,not
1,mpf3x-BjTdTEA3yCZrAYPw,theupsstore,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",49886.0,1546.6,49225,300,mega
2,tUFrWirKiKi_TAnsVWINQQ,target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",41369.0,1853.8,17418,123,mega
3,MTSW4McQd7CbVtyjqoe9mw,sthonorepastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",14984.0,10634.5,59706,1,not
4,mWMc6_wTdE0EUBKIGXDVfA,perkiomenvalleybrewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",5137.0,117.1,89837,1,not
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150342,c8GjPIOTGVmIemT7j5_SyQ,wildbirdsunlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3...",17047.0,902.4,54579,11,small
150343,_QAMST-NrQobXduilWEqSw,clairesboutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None,17443.0,869.6,53121,4,small
150343,_QAMST-NrQobXduilWEqSw,clairesboutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None,17443.0,869.6,55344,4,small
150344,mtGm22y5c2UHNXDFAjaPNw,cycleryfitnesscenter,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",34663.0,119.2,33598,1,not


In [66]:
# Create and normalize the one-hots for chain.
encoded_df = pd.get_dummies(sjoined_df, columns=['chain'])
encoded_df['chain_not'] = encoded_df['chain_not'].astype(int)
encoded_df['chain_small'] = encoded_df['chain_small'].astype(int)
encoded_df['chain_medium'] = encoded_df['chain_medium'].astype(int)
encoded_df['chain_large'] = encoded_df['chain_large'].astype(int)
encoded_df['chain_mega'] = encoded_df['chain_mega'].astype(int)
encoded_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,hours,population,pop_density,median_income,locations,chain_not,chain_small,chain_medium,chain_large,chain_mega
0,Pns2l4eNsfO8kk83dixA6A,abbyrappoportlaccmq,"1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,None,31439.0,3790.5,57604,1,1,0,0,0,0
1,mpf3x-BjTdTEA3yCZrAYPw,theupsstore,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",49886.0,1546.6,49225,300,0,0,0,0,1
2,tUFrWirKiKi_TAnsVWINQQ,target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",41369.0,1853.8,17418,123,0,0,0,0,1
3,MTSW4McQd7CbVtyjqoe9mw,sthonorepastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",14984.0,10634.5,59706,1,1,0,0,0,0
4,mWMc6_wTdE0EUBKIGXDVfA,perkiomenvalleybrewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,"{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",5137.0,117.1,89837,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150342,c8GjPIOTGVmIemT7j5_SyQ,wildbirdsunlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,...,"{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3...",17047.0,902.4,54579,11,0,1,0,0,0
150343,_QAMST-NrQobXduilWEqSw,clairesboutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,...,None,17443.0,869.6,53121,4,0,1,0,0,0
150343,_QAMST-NrQobXduilWEqSw,clairesboutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,...,None,17443.0,869.6,55344,4,0,1,0,0,0
150344,mtGm22y5c2UHNXDFAjaPNw,cycleryfitnesscenter,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,...,"{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",34663.0,119.2,33598,1,1,0,0,0,0


In [57]:
encoded_exploration = encoded_df[encoded_df['locations'] == 60]
encoded_exploration

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,hours,population,pop_density,median_income,locations,chain_not,chain_small,chain_medium,chain_large,chain_mega
998,TwIouh_gbW9TCSK3PhuPrg,tjmaxx,545 Cool Springs Blvd,Franklin,TN,37067,35.948462,-86.823777,3.0,28,...,None,32417.0,384.7,61448,60,False,False,False,True,False
1411,he6ypFmnUF95PlNUWT6i5g,tjmaxx,1130 Market St,Philadelphia,PA,19107,39.951626,-75.159040,3.0,6,...,"{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ...",14984.0,10634.5,59706,60,False,False,False,True,False
4346,FBEzhE4ABeiZTGGO-BTKeQ,phillypretzelfactory,5742 Lower York Rd,New Hope,PA,18938,40.347018,-75.030699,3.0,5,...,"{'Monday': '9:0-16:0', 'Tuesday': '9:0-16:0', ...",14115.0,133.3,300000,60,False,False,False,True,False
4491,DlVUt17R8yfmJ3Zguw-f2Q,phillypretzelfactory,617 Stokes Rd,Medford,NJ,8055,39.872880,-74.811207,3.0,9,...,None,28519.0,275.5,101039,60,False,False,False,True,False
6406,k6TuMo8DLW7RG9xCoo0mrg,tjmaxx,1301 Missouri Ave N,Largo,FL,33770,27.930217,-82.788384,4.0,6,...,"{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3...",25880.0,1921.8,30731,60,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141800,CROa0Os9D_hwjv1O2Vf11w,phillypretzelfactory,1001 Germantown Pike,Plymouth Meeting,PA,19462,40.128459,-75.299797,4.0,16,...,"{'Monday': '6:0-18:0', 'Tuesday': '6:0-18:0', ...",15928.0,740.9,117665,60,False,False,False,True,False
145072,tv6Ac4nKcuTJjsUIfDH2Dg,tjmaxx,2105 66th St N,St. Petersburg,FL,33710,27.787957,-82.727287,4.0,7,...,"{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3...",34462.0,1668.9,49797,60,False,False,False,True,False
146048,qa63e0YXdfhInm7NvdHa6g,tjmaxx,1046 S Kirkwood Rd,Kirkwood,MO,63122,38.564222,-90.404238,3.0,19,...,"{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3...",41336.0,1102.1,18485,60,False,False,False,True,False
149376,uaBIJAmgeeK30m7QRTibtw,phillypretzelfactory,1836 Old York Rd,Abington,PA,19001,40.136295,-75.114607,4.0,8,...,None,17690.0,2134.6,90766,60,False,False,False,True,False
